<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/Advanced/CollectReinforcementData.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install aitk --quiet

In [ ]:
import aitk.robots
aitk.robots.__version__

### Collect wall following data

Create a world with several obstacles.  Use a controller to follow walls on the robot's left
side.  Collect the robot's IR data and movements so as to eventually be able to train a neural
network to learn how to wall follow. Save the data to a CSV file.

In [ ]:
world = aitk.robots.World(width=200, height=200)
world.add_wall("blue", 0, 0, 50, 50)
world.add_wall("blue", 75, 200, 125, 150)
robot = aitk.robots.Scribbler()
world.add_robot(robot)
robot.add_device(aitk.robots.RangeSensor(width=45,max=20,name="front"))
robot.add_device(aitk.robots.RangeSensor(width=45,max=20,position=(6,-6),
                                         a=90,name="front-left"))
robot.add_device(aitk.robots.RangeSensor(width=45,max=20,position=(-6,-6),
                                         a=90,name="back-left"))
robot.set_pose(100,100,0)

In [ ]:
world.watch()

In [ ]:
import random

def scale(v, max):
    return v/max

def controller(robot):
    f = robot["front"].get_distance()
    fl = robot["front-left"].get_distance()
    bl = robot["back-left"].get_distance()
    max_dist = robot["front"].get_max()
    readings = [f, fl, bl]
    inputs = [scale(v, max_dist) for v in readings]
    tolerance = 1.5
    difference = fl - bl
    if robot.stalled:
        situation = "stalled"
        robot.state["stalled"] += 1
    elif f < max_dist:
        # wall in front, turn right
        situation = "blocked"
    elif fl < max_dist or bl < max_dist:
        # left side is against a wall
        if abs(difference) > tolerance and difference < 0:
            # keep left side aligned with wall
            situation = "adjustR"
        elif abs(difference) > tolerance and difference > 0:
            # keep left side aligned with wall
            situation = "adjustL"
        elif min(fl, bl) < 10:
            # robot is too close to wall, move right
            situation = "close"
        elif max(fl, bl) > 15:
            # robot is too far from wall, move left
            situation = "far"
        else:
            # alignment is good, go straight
            situation = "follow"
    else:
        # no wall sensed, move forward to find a wall
        situation = "no_wall"

    if situation == robot.state["situation"]:
        translate, rotate = robot.get_velocity(target=True)
    else:
        robot.state["situation"] = situation
        translate = 1 - random.random() * 2
        rotate = 1 - random.random() * 2
    robot.speak(situation)
    robot.move(translate, rotate)
    targets = [translate, rotate, situation]
    data.append(inputs + targets)


### Test it out

In [ ]:
data = []
robot.set_random_pose()
robot.move(1, 0)
world.seconds(5, [controller])

### Try several random starting points

Collect data from many different random poses so as to create a good variety of data.

In [ ]:
%%time
from aitk.robots.utils import distance

command_sequence = []
TEST_TIME = 10 # in seconds
TOTAL_TIME = 30 * 60 # in seconds
world.time = 0.0

for i in range(TOTAL_TIME // TEST_TIME): 
    data = []
    robot.set_random_pose()
    robot.move(1, 0)
    robot.state["stalled"] = 0
    start_pose = robot.get_pose()
    world.seconds(TEST_TIME, [controller], real_time=False, quiet=True)
    stop_pose = robot.get_pose()
    total_distance = distance(start_pose[0], start_pose[1], stop_pose[0], stop_pose[1])
    score = total_distance - robot.state["stalled"] * 100
    command_sequence.append([data, score])
    

### Sort by total distance traveled

In [ ]:
command_sequence = sorted(command_sequence, key=lambda item: item[1], reverse=True)

In [ ]:
print("Best score:", command_sequence[0][-1])
print("Worst score:", command_sequence[-1][-1])

In [ ]:
good_sequence = [sequence for (sequence, score) in command_sequence if score > 0]
len(good_sequence)

In [ ]:
good_sequence[0]

In [ ]:
data = []
for sequence in good_sequence:
    data.extend(sequence)

In [ ]:
len(data)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data, columns=['F_ir', 'FL_ir', 'BL_ir', 'translate', 'rotate', 'situation'])

In [ ]:
df['situation'].value_counts()

In [ ]:
df.to_csv("unsupervised_data.csv")